In [1]:
# modified from https://github.com/pytorch/examples/blob/master/imagenet/main.py
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets


/home/usami/miniconda3/envs/human_action/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/usami/miniconda3/envs/human_action/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c107WarningC1ENS_7variantIJNS0_11UserWarningENS0_18DeprecationWarningEEEERKNS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEEb'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
#imports
import pandas as pd
import numpy as np
# from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

df_unsw_normal = pd.read_csv('/home/usami/Protocol-Based-Deep-Intrusion-Detection-for-DoS-Normal-and-DDoS-Attacks/normal_ddos_dos_classification/dataset/train/unsw_normal.csv')
df_bot_dos = pd.read_csv('/home/usami/Protocol-Based-Deep-Intrusion-Detection-for-DoS-Normal-and-DDoS-Attacks/normal_ddos_dos_classification/dataset/train/bot_iot_dos.csv')
df_bot_ddos = pd.read_csv('/home/usami/Protocol-Based-Deep-Intrusion-Detection-for-DoS-Normal-and-DDoS-Attacks/normal_ddos_dos_classification/dataset/train/bot_iot_ddos.csv')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def _ternary(x: torch.Tensor, delta: float):
    return (x >= delta).float() - (x <= -delta).float()


class _ternary_without_scale(torch.autograd.Function):
    @staticmethod
    def forward(ctx, *inputs) -> torch.Tensor:
        input_f, running_delta, delta, momentum, training = inputs
        if momentum > 0:
            if training:
                ctx.delta = input_f.norm(1).item() * (delta / input_f.numel())  # = delta * |input_f|_1 / n
                running_delta.data = momentum * ctx.delta + (1.0 - momentum) * running_delta.data
            else:
                ctx.delta = running_delta.data.item()
        else:
            ctx.delta = delta
        input_t = _ternary(input_f, ctx.delta)
        ctx.save_for_backward(input_f)
        return input_t

    @staticmethod
    def backward(ctx, *grad_outputs):
        grad_output, = grad_outputs
        input_f, = ctx.saved_tensors
        grad_input = grad_output * (-1 <= input_f & input_f <= 1).float()
        return grad_input, None, None, None, None, None, None, None, None, None


class _ternary_py(torch.autograd.Function):
    @staticmethod
    def ternary_backward(grad_output: torch.Tensor, x: torch.Tensor, delta: float, order: int, threshold: float):
        scale = 2 * delta
        assert threshold <= scale
        tmp = torch.zeros_like(grad_output)
        tmp += ((x >= -threshold) & (x <= threshold)).float() * order * \
               (torch.fmod(x / delta + 3, 2) - 1).abs().pow(order - 1)
        return grad_output * tmp

    @staticmethod
    def forward(ctx, *inputs) -> torch.Tensor:
        input_f, running_delta, delta, momentum, training, ctx.order = inputs
        if momentum > 0:
            if training:
                ctx.delta = input_f.norm(1).item() * (delta / input_f.numel())  # = delta * |input_f|_1 / n
                running_delta.data = momentum * ctx.delta + (1.0 - momentum) * running_delta.data
            else:
                ctx.delta = running_delta.data.item()
        else:
            ctx.delta = delta
        input_t = _ternary(input_f, ctx.delta) * (2 * ctx.delta)
        ctx.save_for_backward(input_f)
        return input_t

    @staticmethod
    def backward(ctx, *grad_outputs):
        grad_output, = grad_outputs
        input_f, = ctx.saved_tensors
        grad_input = _ternary_py.ternary_backward(grad_output, input_f, ctx.delta, ctx.order, 2. * ctx.delta)
        return grad_input, None, None, None, None, None, None, None, None, None


def ternary(input_f: torch.Tensor, running_delta, delta, momentum, training, order, use_scale=True):
    if not use_scale:
        return _ternary_without_scale.apply(input_f, running_delta, delta, momentum, training)
    else:
        return _ternary_py.apply(input_f, running_delta, delta, momentum, training, order)


class Ternary(torch.nn.Module):
    def __init__(self, config: dict, *arg, **kwargs):
        super(Ternary, self).__init__()
        self.config = config
        self.delta = config.setdefault("delta", 0.5)
        self.momentum = config.setdefault("momentum", 0.01)
        self.track_running_stats = config.setdefault("track_running_stats", True)
        self.order = config.setdefault('order', 2)
        self.use_scale = config.setdefault('use_scale', True)
        assert self.momentum <= 1 and self.order > 0 and self.delta > 0
        self.register_buffer("running_delta", torch.zeros(1))
        self.reset_parameters()

    def reset_parameters(self):
        if self.momentum > 0:
            self.running_delta.fill_(self.delta * 0.7979)
        else:
            self.running_delta.fill_(self.delta)

    def forward(self, input_f):
        return ternary(input_f, self.running_delta, self.delta, self.momentum,
                       self.training and self.track_running_stats, self.order, self.use_scale)

    def extra_repr(self):
        return ", ".join(["{}={}".format(k, v) for k, v in self.config.items()])

In [3]:
df_combined = pd.concat([df_unsw_normal, df_bot_dos, df_bot_ddos], ignore_index=True)

x = df_combined.loc[:, df_combined.columns != 'category']
# x = (x-x.min())/(x.max()-x.min())

df_combined.category = pd.factorize(df_combined.category)[0]
y = df_combined['category']

scaler = StandardScaler()
x = scaler.fit_transform(x)
for i in [0,1,2,3,7,8]:
    x[:,i] = np.where(x[:,i] > 0, np.log(x[:,i]), 0)
    
x_label_data = torch.tensor(x, dtype=torch.float).to(device)
y_label_data = torch.tensor(y, dtype=torch.long).to(device)

dataset = TensorDataset(x_label_data, y_label_data)
train_size = int(0.7 * len(dataset))
valid_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

# Create data loaders
batch_size = 8192
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

/tmp/ipykernel_12712/4249798583.py:12: RuntimeWarning: invalid value encountered in log
  x[:,i] = np.where(x[:,i] > 0, np.log(x[:,i]), 0)


In [152]:
class BinaryActivationFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # Binarize input to -1 or 1
        output = input.sign()
        output[output==0] = 1
        ctx.save_for_backward(input)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input.abs() > 1] = 0
        return grad_input

class BinaryActivation(nn.Module):
    def forward(self, input):
        return BinaryActivationFunction.apply(input)

class QLinear(torch.nn.Linear):
    qa_config = {}
    qw_config = {}

    def __init__(self, in_features, out_features, bias=False):
        super().__init__(in_features, out_features, bias)
        # self.input_quantizer = BinaryActivation()
        self.weight_quantizer = BinaryActivation()

    def forward(self, input_f):
        # input_t = self.input_quantizer(input_f)
        weight_b = self.weight_quantizer(self.weight)
        out = F.linear(input_f, weight_b, self.bias)
        return out

In [160]:
class Netb(nn.Module):
    def __init__(self):
        super(Netb, self).__init__()
        self.input_quantizer = BinaryActivation()

        self.linear1 = QLinear(13, 128, bias = False)
        self.bn1 = nn.BatchNorm1d(128)
        # self.i1 = nn.Identity()

        self.linear2 = QLinear(128, 128, bias = False)
        self.bn2 = nn.BatchNorm1d(128)
        # self.i2 = nn.Identity()

        self.linear3 = QLinear(128, 128, bias = False)
        self.bn3 = nn.BatchNorm1d(128)
        # self.i3 = nn.Identity()

        self.linear4 = QLinear(128, 3, bias = False)
        self.act4 = nn.Softmax(dim = 1)

    def forward(self, x):
        x = self.input_quantizer(x)
        x = self.linear1(x)

        x = self.input_quantizer(x)
        x = self.linear2(x)
        # x = self.bn2(x)
        # x = self.i2(x)
        x = self.input_quantizer(x)
        x = self.linear3(x)
        # x = self.bn3(x)
        # x = self.i3(x)
        x = self.input_quantizer(x)
        x = self.linear4(x)
        x = self.act4(x)
        return x
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_b = Netb().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_b.parameters(), 0.01,
                                momentum=0.9,
                                weight_decay=1e-4)
num_epochs = 5
for epoch in range(num_epochs):
    model_b.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_b(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # model_ter1.set_weights_and_biases()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}', end='\r')
    
    model_b.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_b(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%')

# Final evaluation on the test set
model_b.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_b(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

torch.save(model_b.state_dict(), 'saved_best_model_bnn.pt')

Epoch [1/5], Validation Accuracy: 75.11%4
Epoch [2/5], Validation Accuracy: 92.33%3
Epoch [3/5], Validation Accuracy: 92.15%4
Epoch [4/5], Validation Accuracy: 89.12%4
Epoch [5/5], Validation Accuracy: 92.27%6
Test Accuracy: 92.16%


In [277]:
class BinaryActivationFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # Binarize input to -1 or 1
        output = input.sign()
        output[output==0] = 1
        ctx.save_for_backward(input)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input.abs() > 1] = 0
        return grad_input

class BinaryActivation(nn.Module):
    def forward(self, input):
        return BinaryActivationFunction.apply(input)

class QLinear(torch.nn.Linear):
    qa_config = {}
    qw_config = {}
    def __init__(self, in_features, out_features, bias=True):
        super().__init__(in_features, out_features, bias)
        self.weight_quantizer = BinaryActivation()
    def forward(self, input_f):
        weight_b = self.weight_quantizer(self.weight)
        out = F.linear(input_f, weight_b, self.bias)
        return out

class Netb(nn.Module):
    def __init__(self):
        super(Netb, self).__init__()
        self.input_quantizer = BinaryActivation()
        self.linear1 = QLinear(13, 128, bias = False)

        self.linear2 = QLinear(128, 128, bias = False)

        self.linear3 = QLinear(128, 128, bias = False)

        self.linear4 = QLinear(128, 3, bias = False)
        self.act4 = nn.Softmax(dim = 1)

    def forward(self, x):
        x = self.input_quantizer(x)
        x = self.linear1(x)

        x = self.input_quantizer(x)
        x = self.linear2(x)

        x = self.input_quantizer(x)
        x = self.linear3(x)

        x = self.input_quantizer(x)
        x = self.linear4(x)
        # x = self.act4(x)
        return x

model_bnn = Netb().to(device)
model_bnn.linear1.state_dict()['weight'].copy_(model_b.linear1.state_dict()['weight'].sign())
model_bnn.linear2.state_dict()['weight'].copy_(model_b.linear2.state_dict()['weight'].sign())
model_bnn.linear3.state_dict()['weight'].copy_(model_b.linear3.state_dict()['weight'].sign())
model_bnn.linear4.state_dict()['weight'].copy_(model_b.linear4.state_dict()['weight'].sign())

tensor([[-1., -1., -1., -1., -1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,
         -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,
          1., -1., -1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,
         -1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,  1., -1.,
          1., -1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,
          1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
         -1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1.,  1., -1.,  1.,  1.,
         -1., -1.,  1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1.,  1., -1.,
         -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,
          1., -1.],
        [ 1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1.,
          1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1., -1.,  1.,  1.,
          1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,
         -1., -1., -1.,  1., -1.

In [278]:
model_bnn.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_bnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')
torch.save(model_bnn.state_dict(), 'saved_best_model_bnn_1.pt')

Test Accuracy: 92.16%


In [272]:
def convert_and_concatenate(arr):
    # Chuyển đổi -1 thành 1 và 1 thành 0
    arr = torch.flip(arr, dims=[0])
    converted_arr = [1 if x == -1 else 0 for x in arr]
    # Ghép các phần tử lại thành một chuỗi
    concatenated_str = ''.join(map(str, converted_arr))

    current_length = len(concatenated_str)
    total_length = (np.ceil(current_length/32)*32).astype(int)
    if current_length < total_length:
        padding_length = int(total_length - current_length)
        concatenated_str = '0' * padding_length + concatenated_str
    # Chuyển chuỗi thành số nhị phân và sau đó thành số hex
    hex_value = hex(int(concatenated_str, 2))[2:]  # Bỏ tiền tố '0x'

    # Đảm bảo chuỗi hex đủ độ dài cần thiết
    hex_length = total_length // 4  # 1 hex digit = 4 bits
    if len(hex_value) < hex_length:
        hex_value = '0' * (hex_length - len(hex_value)) + hex_value
    return hex_value

def save_model_parameters_to_txt(model, file_path):
    with open(file_path, 'w') as f:
        for name, param in model.named_parameters():
            f.write(f'{name}\n')
            f.write(f'input_channel: {param.shape[1]}\n')
            f.write(f'output_channel: {param.shape[0]}\n')
            for param_e in param:
                hex_conv = convert_and_concatenate(param_e)
                segments = [hex_conv[i:i+8] for i in range(0, len(hex_conv), 8)]
                segments.reverse()
                for segment in segments:
                    f.write(f'0x{segment}\n')
            f.write(f'\n')
    return 0

# Gọi hàm để lưu thông số mô hình
a = save_model_parameters_to_txt(model_bnn, 'model_parameters.txt')

In [121]:
model_bnn.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_bnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Accuracy: 91.12%


In [284]:
model_bnn.cpu()

Netb(
  (input_quantizer): BinaryActivation()
  (linear1): QLinear(
    in_features=13, out_features=128, bias=False
    (weight_quantizer): BinaryActivation()
  )
  (linear2): QLinear(
    in_features=128, out_features=128, bias=False
    (weight_quantizer): BinaryActivation()
  )
  (linear3): QLinear(
    in_features=128, out_features=128, bias=False
    (weight_quantizer): BinaryActivation()
  )
  (linear4): QLinear(
    in_features=128, out_features=3, bias=False
    (weight_quantizer): BinaryActivation()
  )
  (act4): Softmax(dim=1)
)

In [285]:
for i in range(10000):
    outputs = model_bnn(images.cpu())

In [302]:
class Net_fp(nn.Module):
    def __init__(self):
        super(Net_fp, self).__init__()
        self.linear1 = nn.Linear(13, 128, bias = False)
        # self.bn1 = nn.BatchNorm1d(128)
        self.act1 = nn.ReLU()

        self.linear2 = nn.Linear(128, 128, bias = False)
        # self.bn2 = nn.BatchNorm1d(128)
        self.act2 = nn.ReLU()


        self.linear3 = nn.Linear(128, 128, bias = False)
        # self.bn3 = nn.BatchNorm1d(128)
        self.act3 = nn.ReLU()

        self.linear4 = nn.Linear(128, 3, bias = False)
        self.act4 = nn.Softmax(dim = 1)

    def forward(self, x):

        x = self.linear1(x)
        # x = self.bn1(x)
        x = self.act1(x)

        x = self.linear2(x)
        # x = self.bn2(x)
        x = self.act2(x)

        x = self.linear3(x)
        # x = self.bn3(x)
        x = self.act3(x)

        x = self.linear4(x)
        x = self.act4(x)
        return x
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_fp = Net_fp().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_fp.parameters(), 0.01,
                                momentum=0.9,
                                weight_decay=1e-4)
num_epochs = 2
for epoch in range(num_epochs):
    model_fp.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_fp(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # model_ter1.set_weights_and_biases()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}', end='\r')
    
    model_fp.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_fp(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%')

# Final evaluation on the test set
model_fp.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model_fp(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

torch.save(model_fp.state_dict(), 'saved_best_model_fp.pt')

Epoch [1/2], Validation Accuracy: 91.52%1
Epoch [2/2], Validation Accuracy: 91.63%0
Test Accuracy: 91.51%


In [303]:
def save_model_fp_parameters_to_txt(model, file_path):
    with open(file_path, 'w') as f:
        for name, param in model.named_parameters():
            f.write(f'{name}\n')
            if('linear' in name and 'weight'in name):
                f.write(f'input_channel: {param.shape[1]}\n')
                f.write(f'output_channel: {param.shape[0]}\n')
            f.write(f'{param}\n')
            f.write(f'\n')
    return 0

# Gọi hàm để lưu thông số mô hình
a = save_model_fp_parameters_to_txt(model_fp, 'fp_model_parameters.txt')

In [293]:
model_fp.bn1.state_dict()

OrderedDict([('weight',
              tensor([0.9876, 0.9864, 0.9845, 0.9805, 1.0014, 0.9878, 0.9857, 0.9918, 0.9910,
                      0.9740, 0.9809, 0.9821, 0.9844, 0.9829, 0.9852, 0.9847, 0.9843, 0.9849,
                      0.9867, 0.9938, 0.9877, 0.9874, 0.9955, 0.9818, 0.9804, 0.9836, 0.9967,
                      0.9887, 0.9758, 0.9847, 0.9884, 0.9849, 0.9843, 0.9887, 0.9859, 1.0009,
                      0.9802, 1.0014, 0.9846, 0.9846, 0.9868, 0.9878, 0.9842, 1.0002, 0.9851,
                      0.9809, 0.9777, 0.9897, 0.9965, 0.9884, 0.9831, 0.9852, 0.9786, 0.9919,
                      0.9769, 0.9830, 0.9811, 0.9776, 0.9879, 0.9862, 0.9808, 0.9792, 0.9844,
                      0.9930, 0.9875, 0.9875, 0.9828, 0.9937, 0.9916, 0.9778, 0.9867, 0.9819,
                      0.9794, 0.9804, 0.9839, 0.9921, 0.9869, 0.9823, 0.9920, 0.9908, 0.9864,
                      0.9843, 0.9846, 0.9862, 0.9877, 0.9890, 0.9839, 0.9883, 0.9829, 0.9843,
                      0.9796, 0.9813

In [309]:
def bit_count(n):
    n = (n & 0x55555555) + ((n >> 1) & 0x55555555)
    n = (n & 0x33333333) + ((n >> 2) & 0x33333333)
    n = (n & 0x0f0f0f0f) + ((n >> 4) & 0x0f0f0f0f)
    n = (n & 0x00ff00ff) + ((n >> 8) & 0x00ff00ff)
    n = (n & 0x0000ffff) + ((n >> 16) & 0x0000ffff)
    return n

# Ví dụ sử dụng:
n = 0b11111111101111000011111111111100  # 13 trong hệ thập phân
print(bit_count(n))  # Kết quả sẽ là 3

25
